# Tiền xử lý

In [7]:
import pandas as pd
import datetime as dt
import os
import numpy as np

* Sau khi xử lý ở mỗi hàng của dữ liệu sẽ bao gồm 3 ngày.  
* Dữ liệu 2 ngày đầu sẽ được dùng làm input để dự báo ngày tiếp theo.

In [8]:
# Gom dữ liệu, 3 ngày liên tiếp là 1 hàng, 2 ngày đầu có đủ còn ngày cuối cùng chỉ có cột cần dự đoán.
def get_dataset(df):
    data = []
    n = df.shape[0]
    
    for i in range(n - 2):
        # Kiểm tra xem 2 ngày tiếp có tồn tại không.
        if df.iloc[i]['date'] + dt.timedelta(days=1) != df.iloc[i + 1]['date']:
            continue
        if df.iloc[i]['date'] + dt.timedelta(days=2) != df.iloc[i + 2]['date']:
            continue
        
        x = {}
        x['temp1'] = df.iloc[i]['temp']
        x['temp2'] = df.iloc[i + 1]['temp']
        x['dewPt1'] = df.iloc[i]['dewPt']
        x['dewPt2'] = df.iloc[i + 1]['dewPt']
        x['press1'] = df.iloc[i]['pressure']
        x['press2'] = df.iloc[i + 1]['pressure']
        x['humid1'] = df.iloc[i]['rh']
        x['humid2'] = df.iloc[i + 1]['rh']
        x['wdir1'] = df.iloc[i]['wdir']
        x['wdir2'] = df.iloc[i + 1]['wdir']
        x['wspeed1'] = df.iloc[i]['wspd']
        x['wspeed2'] = df.iloc[i + 1]['wspd']
        
        x['temp_min1'] = df.iloc[i]['temp_min']
        x['temp_min2'] = df.iloc[i + 1]['temp_min']
        x['dewPt_min1'] = df.iloc[i]['dewPt_min']
        x['dewPt_min2'] = df.iloc[i + 1]['dewPt_min']
        x['press_min1'] = df.iloc[i]['pressure_min']
        x['press_min2'] = df.iloc[i + 1]['pressure_min']
        x['humid_min1'] = df.iloc[i]['rh_min']
        x['humid_min2'] = df.iloc[i + 1]['rh_min']
        x['wdir_min1'] = df.iloc[i]['wdir_min']
        x['wdir_min2'] = df.iloc[i + 1]['wdir_min']
        x['wspeed_min1'] = df.iloc[i]['wspd_min']
        x['wspeed_min2'] = df.iloc[i + 1]['wspd_min']
        
        x['temp_max1'] = df.iloc[i]['temp_max']
        x['temp_max2'] = df.iloc[i + 1]['temp_max']
        x['dewPt_max1'] = df.iloc[i]['dewPt_max']
        x['dewPt_max2'] = df.iloc[i + 1]['dewPt_max']
        x['press_max1'] = df.iloc[i]['pressure_max']
        x['press_max2'] = df.iloc[i + 1]['pressure_max']
        x['humid_max1'] = df.iloc[i]['rh_max']
        x['humid_max2'] = df.iloc[i + 1]['rh_max']
        x['wdir_max1'] = df.iloc[i]['wdir_max']
        x['wdir_max2'] = df.iloc[i + 1]['wdir_max']
        x['wspeed_max1'] = df.iloc[i]['wspd_max']
        x['wspeed_max2'] = df.iloc[i + 1]['wspd_max']
        
        x['rain1'] = df.iloc[i]['rain']
        x['rain2'] = df.iloc[i + 1]['rain']
        x['is_rain1'] = df.iloc[i]['is_rain']
        x['is_rain2'] = df.iloc[i + 1]['is_rain']
        
        x['y_temp'] = df.iloc[i + 2]['temp']
        x['y_temp_min'] = df.iloc[i + 2]['temp_min']
        x['y_temp_max'] = df.iloc[i + 2]['temp_max']
        x['y_rain'] = df.iloc[i + 2]['rain']
        x['y_is_rain'] = df.iloc[i + 2]['is_rain']
        data.append(x)

    return data

* Cột thời tiết được cho là có mưa nếu chứa 1 trong những từ khóa: ['Shower', 'Drizzle', 'Rain', 'T-Storm', 'Thunder'].

In [9]:
def is_rain(weather):
    for x in ['Shower', 'Drizzle', 'Rain', 'T-Storm', 'Thunder']:
        if x in weather:
            return 1
    return 0

* Do dữ liệu thời tiết liên tục, thay đổi khá chậm và dữ liệu thiếu rất ít nên các dữ liệu thiếu sẽ được thay thế bởi dữ liệu gần nó.  
* Dữ liệu thuộc tính sẽ đưa về dạng số.  
* Cột thời tiết sẽ được đổi thành có mưa hoặc không thông qua hàm 'is_rain(weather)'.  
* Các hàng dữ liệu sẽ được gom lại theo ngày và lấy max, mean, min. Riêng với cột thời tiết sẽ tính là 1 nếu có 1 lần trong ngày mưa và cột 'rain' là tỉ lệ mưa trong 1 ngày.

In [10]:
def preprocess(df):
    # Chọn các cột cần lấy
    df = df[['valid_time_gmt', 'temp', 'dewPt', 'pressure', 'rh', 'wx_phrase','wdir_cardinal', 'wspd']]
    # Lấp dữ liệu thiếu
    df = df.replace(0, np.nan)
    df = df.ffill()
    df = df.bfill()
    
    # Đổi hướng gió thành số
    df['wdir_cardinal'] = df['wdir_cardinal'].astype('category').cat.codes
    # Đổi thời tiết thành có mưa hay không(1 là có)
    df['wx_phrase'] = df['wx_phrase'].apply(lambda x: is_rain(x))
    # Đổi múi giờ về Việt Nam
    df['valid_time_gmt'] = pd.to_datetime(df['valid_time_gmt'], unit='s').apply(
        lambda x: x.tz_localize('GMT').tz_convert('Asia/Ho_Chi_Minh') 
    )
    df['date'] = df['valid_time_gmt'].dt.date
    
    # Gom dữ liệu theo ngày, lấy max, min, mean
    group = df.groupby('date')
    df = pd.DataFrame()
    df['temp'] = group['temp'].mean().round(1)
    df['temp_min'] = group['temp'].min().round(1)
    df['temp_max'] = group['temp'].max().round(1)
    df['dewPt'] = group['dewPt'].mean().round(1)
    df['dewPt_min'] = group['dewPt'].min().round(1)
    df['dewPt_max'] = group['dewPt'].max().round(1)
    df['pressure'] = group['pressure'].mean().round(2)
    df['pressure_min'] = group['pressure'].min().round(2)
    df['pressure_max'] = group['pressure'].max().round(2)
    df['rh'] = group['rh'].mean().round(1)
    df['rh_min'] = group['rh'].min().round(1)
    df['rh_max'] = group['rh'].max().round(1)
    df['wdir'] = group['wdir_cardinal'].mean().round(0)
    df['wdir_min'] = group['wdir_cardinal'].min().round(0)
    df['wdir_max'] = group['wdir_cardinal'].max().round(0)
    df['wspd'] = group['wspd'].mean().round(0)
    df['wspd_min'] = group['wspd'].min().round(0)
    df['wspd_max'] = group['wspd'].max().round(0)
    # Tỉ lệ mưa
    df['rain'] = group['wx_phrase'].mean().round(2)
    # Trong ngày có mưa không?
    df['is_rain'] = group['wx_phrase'].max()
    return df.reset_index()

In [11]:
if not os.path.isdir('c_data'):
    os.mkdir('c_data')
    
files = os.listdir('data')

df = pd.read_csv(f'data/data.csv')
    
df = preprocess(df)

dataset = get_dataset(df)
df = pd.DataFrame(dataset)
df.to_csv(f'c_data/data.csv', index=False)

* Dữ liệu cuối cùng để đưa vào model

In [12]:
df

,temp1,temp2,dewPt1,dewPt2,press1,press2,humid1,humid2,wdir1,wdir2,...,wspeed_max2,rain1,rain2,is_rain1,is_rain2,y_temp,y_temp_min,y_temp_max,y_rain,y_is_rain
0,26.5,27.1,21.0,21.0,1007.85,1008.36,74.6,70.6,9.0,6.0,...,15.0,0.0,0.0,0,0,27.2,24.0,32.0,0.0,0
1,27.1,27.2,21.0,21.9,1008.36,1008.59,70.6,73.4,6.0,8.0,...,19.0,0.0,0.0,0,0,27.4,23.0,32.0,0.0,0
2,27.2,27.4,21.9,20.0,1008.59,1007.80,73.4,65.2,8.0,4.0,...,17.0,0.0,0.0,0,0,27.0,23.0,33.0,0.0,0
3,27.4,27.0,20.0,21.8,1007.80,1007.95,65.2,74.7,4.0,6.0,...,17.0,0.0,0.0,0,0,26.5,23.0,33.0,0.0,0
4,27.0,26.5,21.8,21.3,1007.95,1008.61,74.7,74.8,6.0,6.0,...,28.0,0.0,0.0,0,0,26.7,21.0,33.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3633,27.2,27.2,20.9,22.2,1009.24,1009.80,70.1,75.7,11.0,11.0,...,15.0,0.0,0.0,0,0,27.0,23.0,33.0,0.0,0
3634,27.2,27.0,22.2,21.0,1009.80,1009.78,75.7,72.5,11.0,10.0,...,17.0,0.0,0.0,0,0,26.2,23.0,31.0,0.0,0
3635,27.0,26.2,21.0,20.5,1009.78,1010.34,72.5,72.7,10.0,12.0,...,13.0,0.0,0.0,0,0,26.6,21.0,32.0,0.0,0
3636,26.2,26.6,20.5,21.0,1010.34,1009.95,72.7,73.9,12.0,11.0,...,17.0,0.0,0.0,0,0,28.1,24.0,33.0,0.0,0
